# Ray Tune Tutorial - 01: Understanding Hyperparameter Tuning - Exercise Solution

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

Unlike for previous tutorials, we use separate notebooks for the Tune tutorial solutions, because most of the exercises take a long time to run.

## Exercise - Try More Neural Network Sizes

Repeat the experiment above using the sizes `[20, 40, 60, 80, 100]` or some subset of these numbers, depending on how long you are willing to wait. What combination appears to be best, given the considerations we discussed above?

First, we set up everything we need from the lesson.

In [ ]:
import ray
from ray import tune

In [ ]:
!../../tools/start-ray.sh --check --verbose

In [ ]:
ray.init(address='auto', ignore_reinit_error=True)

In [ ]:
sizes = [20, 40, 60, 80, 100]

The next cell will take around 20-30 minutes, even on a fast laptop.

In [ ]:
analysis = tune.run(
    "PPO",
    stop={"episode_reward_mean": 400},

    config={
        "env": "CartPole-v1",
        "num_gpus": 0,
        "num_workers": 3,
        "model": {
            'fcnet_hiddens': [
                tune.grid_search(sizes),
                tune.grid_search(sizes)
            ]
        },
        "eager": False,
    },
    verbose=1
)

## Understanding the Results

First, how long did this take?

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

Which one performed best based on our stopping criteria?

In [ ]:
analysis.get_best_config(metric="episode_reward_mean")

Interesting that the best result is for the smallest size for the first layer and the largest size for the second layer, but recall what we said in the lesson about all values providing good results.

In [ ]:
df = analysis.dataframe()
df

Let's sort by `timesteps_total` to see which ones were fastest.

In [ ]:
df.sort_values('timesteps_total', ascending=True)

In [ ]:
df[['episode_reward_mean', 'training_iteration', 'timesteps_total', 'config/model']].sort_values('timesteps_total', ascending=True)

It appears that the largest networks trained the fastest, `[100,100]`, `[60,100]`, and `[80,100]`, followed closely by some smaller configurations. The larger networks would be easier to train, because they more parameters, but the larger parameter sets would increase training times, but apparently not enough to tip the balance against them. 

However, the differences are still relatively small compared to our previous pick of `[40,40]`. If you compare the timestamp values, the training time for `[40,40]` is about 20% slower.

In [ ]:
ray.shutdown()  # "Undo ray.init()".